In [1]:


import matplotlib.pyplot as plt
import panel as pn
import plotly.express as px
from pathlib import Path
from panel.interact import interact
from panel import widgets
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import json
import alpaca_trade_api as tradeapi
import requests
import os
import seaborn as sns
from dotenv import load_dotenv
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

%matplotlib inline



# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')


ImportError: cannot import name 'encode_utf8' from 'bokeh.util.string' (/Users/bulya/opt/anaconda3/envs/vizenv/lib/python3.7/site-packages/bokeh/util/string.py)

In [2]:
# Read the API key

load_dotenv()

APCA_API_KEY_ID = os.getenv("Api_key")
APCA_SECRET_KEY = os.getenv("Secret_key")

api = tradeapi.REST(APCA_API_KEY_ID, APCA_SECRET_KEY, api_version='v2')

type(APCA_API_KEY_ID)



NameError: name 'load_dotenv' is not defined

In [184]:
# Import Data 
materials_sector = api.alpha_vantage.historic_quotes('XLB', adjusted=True, output_format='pandas')
#xlb_df.head()

industrials_sector = api.alpha_vantage.historic_quotes('XLI', adjusted=True, output_format='pandas')
#industrials_sector.head()


# xlk_df = api.alpha_vantage.historic_quotes('XLK', adjusted=True, output_format='pandas')
# # xlk_df.tail()

# rwr_df = api.alpha_vantage.historic_quotes('RWR', adjusted=True, output_format='pandas')
# #rwr_df.head()

# #Use alpha vantage to get historic quotes for S&P500. Will need this later.

# spy_df = api.alpha_vantage.historic_quotes('SPY', adjusted=True, output_format='pandas')

In [185]:
materials_sector.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
materials_sector.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
materials_sector.sort_index(ascending=True, inplace=True)

In [186]:
industrials_sector.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
industrials_sector.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
industrials_sector.sort_index(ascending=True, inplace=True)

In [187]:
# Define Panel Visualization Functions
#Function to create backtesting plot
def backtest_materials():
    class SmaCross(Strategy):
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, 10)
            self.ma2 = self.I(SMA, Close, 20)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_materials = Backtest(materials_sector, SmaCross,
                              cash=10000, commission=.002)
    bt_materials.run()
    bt_materials_plot = bt_materials.plot()
    #return bt_materials_plot
    return bt_materials.plot()


# def create_housing_units_plot():
#     """Housing Units Per Year."""
#     housing_units = sfo_data.groupby('year').mean()
#     housing_units.head(7)

#     housing_units_plot = housing_units['housing_units'].hvplot.bar(
#         ylim = (370000,387000), ylabel = "Housing Units",
#         title = "Housing Units in San Francisco from 2010 to 2016")
    
# #     plt.ylabel('Housing Units')
# #     plt.title("Housing Units in San Francisco from 2010 to 2016", fontsize=14)
    
#     return housing_units_plot  

In [188]:
def backtest_industrials():
    class SmaCross(Strategy):
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, 10)
            self.ma2 = self.I(SMA, Close, 20)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_industrials = Backtest(industrials_sector, SmaCross,
                              cash=10000, commission=.002)
    bt_industrials.run()
    bt_industrials_plot = bt_industrials.plot()
            
    return bt_industrials_plot


In [189]:
# Create panels to structure the layout of the dashboard

column1 = pn.Column(
    "## Materials backtest", backtest_materials(), 
   
)

column2 = pn.Column(
      "## Industrials backtest", backtest_industrials(), 
   
 )

# column3 = pn.Column(
#     "## Top 10 and Neighborhood Map", create_top10_plot(), 
#     create_neighborhood_map()
# )

# column4 = pn.Column(
#     "## Parallel Coordinates and Parallel Categories",
#     create_parallel_coordinates(),
#     create_parallel_categories(),
# )

# Create tabs





sector_dashboard = pn.Tabs(
    ("Materials", column1), ("Industrial", column2)
)  
    
    #("Avg Sales Prices", column2), ("Top 10 Neighborhoods and Map", column3),
    #("Parallels", column4)
    


In [143]:
#Serve the dashboard

sector_dashboard.servable()


Tabs
    [0] Column
        [0] Markdown(str)
        [1] Str(None)
    [1] Column
        [0] Markdown(str)
        [1] Str(None)